## Topographic Complexity/Variability: Roughness  
Developed in November 2023 by Dr. Larry Syu-Heng Lai (University of Washington)  

Recommended reference:
* Wilson, M.F.J., O’Connell, B., Brown, C., Guinan, J.C., Grehan, A.J., 2007. Multiscale Terrain Analysis of Multibeam Bathymetry Data for Habitat Mapping on the Continental Slope. Marine Geodesy 30, 3-35. https://doi.org/10.1080/01490410701295962 

### Initial setup

In [1]:
import numpy as np
import rasterio
from scipy.ndimage import maximum_filter, minimum_filter
from tqdm.notebook import tqdm

### Define data path

In [2]:
# Define your file paths and file names separately
input_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
input_file_name = 'Test_DEM.tif'
#input_file_name = 'Tokeland_DEM.tif'
#input_file_name = 'Nemah_DEM.tif'
#input_file_name = 'Francies_DEM.tif'

output_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
output_file_name = 'Test_pyRoughness.tif'
#output_file_name = 'Tokeland_pyRoughness.tif'
#output_file_name = 'Nemah_pyRoughness.tif'
#output_file_name = 'Francies_pyRoughness.tif'

# Combine folder and file names to create the full paths
input_tif_path = input_folder + input_file_name
output_tif_path = output_folder + output_file_name

### Read a DEM

In [3]:
with rasterio.open(input_tif_path) as src:
    dem = src.read(1)  # Read the first band into a 2D array
    meta = src.meta

See coordinate system info of the GeoTIFF

In [4]:
# Open the GeoTIFF file
with rasterio.open(input_tif_path) as src:
    # Read the CRS
    crs = src.crs
    
    # Print the CRS information
    print(f"CRS: {crs}")
    print(f"CRS as WKT: {crs.wkt}")
    print(f"CRS as PROJ string: {crs.to_proj4()}")
    print(f"CRS as EPSG code: {crs.to_epsg()}")
    print(f"CRS as dictionary: {crs.to_dict()}")

CRS: PROJCS["NAD83(HARN) / Washington South (ftUS)",GEOGCS["NAD83(HARN)",DATUM["North_American_1983_HARN",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",45.3333333333333],PARAMETER["central_meridian",-120.5],PARAMETER["standard_parallel_1",45.8333333333333],PARAMETER["standard_parallel_2",47.3333333333333],PARAMETER["false_easting",1640416.66666667],PARAMETER["false_northing",0],UNIT["US survey foot",0.304800609601219,AUTHORITY["EPSG","9003"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
CRS as WKT: PROJCS["NAD83(HARN) / Washington South (ftUS)",GEOGCS["NAD83(HARN)",DATUM["North_American_1983_HARN",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitud

## Roughness (R)

Roughness (R) is calculated as the difference between the maximum and minimum elevation values within a specified window size surrounding each pixel:

$$
R(n) = B_{\text{max}} - B_{\text{min}}
$$

where:
- $ B_{\text{max}}(n) $ is the maximum elevation in the $ n \times n $ window.
- $ B_{\text{min}}(n) $ is the minimum elevation in the $ n \times n $ window.


##### Rugosity function  

In [5]:
def calculate_roughness(dem, window_size):
    """
    Calculate the Roughness for a digital elevation model (DEM) with progress bar.
    
    :param dem: A 2D numpy array of the DEM.
    :param window_size: The size of the window to calculate Roughness (must be an odd number).
    :return: A 2D numpy array of the Roughness.
    """
    rows, cols = dem.shape
    roughness = np.zeros((rows, cols), dtype=dem.dtype)
    
    # Define the step size for updating the progress bar (could be rows, columns, or a fixed value)
    step = 1 if rows < 100 else rows // 100  # Adjust the step size as needed
    
    with tqdm(total=rows * cols, desc="Calculating Roughness") as pbar:
        for i in range(0, rows, step):
            for j in range(0, cols, step):
                # Define the window boundaries
                row_slice = slice(max(i - window_size // 2, 0), min(i + window_size // 2 + 1, rows))
                col_slice = slice(max(j - window_size // 2, 0), min(j + window_size // 2 + 1, cols))
                
                # Apply the filters to the current window
                window = dem[row_slice, col_slice]
                max_within_window = maximum_filter(window, size=window_size, mode='nearest')
                min_within_window = minimum_filter(window, size=window_size, mode='nearest')
                
                # Calculate roughness and update the corresponding window in the result
                roughness_window = max_within_window - min_within_window
                roughness[row_slice, col_slice] = roughness_window
                
                # Update progress bar after processing each window
                pbar.update(step * step)
    
    # Correct the progress bar if the step size does not evenly divide rows or cols
    pbar.n = rows * cols
    pbar.refresh()

    return roughness

Calculate Roughness with a given window size

In [6]:
# Calculate with a N x N window as an example
window_size = 3  # Replace with the desired window size

# Calculate Roughness
roughness = calculate_roughness(dem, window_size)

Calculating Roughness:   0%|          | 0/1855236 [00:00<?, ?it/s]

Output data into GeoTIFF
* Enabling geotiff compression to reduce writing time
* Enabling Tile-based writing if needed
* Enabling BIGTIFF parameter to allow writing a large GeoTIFF

In [7]:
# Update metadata for output GeoTIFF
meta.update(dtype=rasterio.float32, compress='lzw', tiled=True, bigtiff='IF_SAFER')

# Write Roughness to a new GeoTIFF
with rasterio.open(output_tif_path, 'w', **meta) as dst:
    dst.write(roughness.astype(rasterio.float32), 1)